In [1]:
#import jax.numpy as np
#from jax import pmap
import numpy as np
from maxnorm.maxnorm_completion import *
from maxnorm.tenalg import *
from maxnorm.graphs import *
import sparse
from itertools import product
import networkx as nx

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

%load_ext autoreload

In [2]:
# create random, low-rank tensor
t = 5
n = 30
r = 3
max_iterations = 100
epsilons = np.logspace(-4, 0, 5)
epsilons = np.concatenate((np.array(0), epsilons), axis = None)
#delta = 0.1
const = 24
sigma = 0.005
ndata =  const * r * t * n * np.log10(n)
U = kr_random(n, t, r, rvs='unif')
U = kr_rescale(U, np.sqrt(n**t), 'hs')
norm_true = np.sqrt(kr_dot(U, U) / n**t)
#U = [np.random.randn(n, r) for i in range(t)]
print("n data: %.2e" % ndata)
print("n data ** t/2: %.2e" % int(const * r * n**(t/2) * np.log10(n)))
print("true norm: %.2e" % norm_true)

n data: 1.60e+04
n data ** t/2: 5.24e+05
true norm: 1.00e+00
n data: 1.60e+04
n data ** t/2: 5.24e+05
true norm: 1.00e+00


In [3]:
qnorm_true = max_qnorm_ub(U)
print(qnorm_true)
print(r**(t/2))

52.26640563363148
15.588457268119896
52.26640563363148
15.588457268119896


In [4]:
expander = nx.random_regular_graph(6, n)
#expander = nx.chordal_cycle_graph(n)
observation_mask = obs_mask_expander(expander, t)
#observation_mask = obs_mask_iid(tuple([n for i in range(t)]), ndata * n**(-t))

In [5]:
from run_sweep_iid import generate_data


data = generate_data(observation_mask, U, sigma)
clean_data_rmse = np.sqrt(loss(U, data) / data.nnz)
print(data.nnz)
print(ndata)
print(n**t)
print("%0.1e%%" % (float(data.nnz) / n**t * 100))

38880
15952.909550972354
24300000
1.6e-01%
38880
15952.909550972354
24300000
1.6e-01%


In [6]:
def print_factor_norms(U):
    print("fro:   " + str([np.linalg.norm(Ui,'fro')**2 for Ui in U]))
    print("2-inf: " + str([np.max(np.linalg.norm(Ui, axis=1)) for Ui in U]))

print_factor_norms(kr_balance_factors(U))

fro:   [71.89108760026208, 71.89108760026208, 71.89108760026208, 71.89108760026205, 71.89108760026208]
2-inf: [2.1153896760319384, 2.204962235255038, 2.3065855738385337, 2.254412223485199, 2.2582825891150646]
fro:   [71.89108760026208, 71.89108760026208, 71.89108760026208, 71.89108760026205, 71.89108760026208]
2-inf: [2.1153896760319384, 2.204962235255038, 2.3065855738385337, 2.254412223485199, 2.2582825891150646]


In [7]:
delta = 1.5 * sigma
#np.sqrt(np.sum((data.data - clean_data)**2)) * 2 / np.sqrt(data.nnz) * 1.5

print("rms of data:        %f" % clean_data_rmse)
print("delta parameter:    %f" % delta)
print("rmse of U true:     %f" % np.sqrt(loss(U, data) / data.nnz))

rms of data:        0.004993
delta parameter:    0.007500
rmse of U true:     0.004993
rms of data:        0.004993
delta parameter:    0.007500
rmse of U true:     0.004993


In [8]:
#Uinit = kr_rescale(Unew1, np.sqrt(np.product(data.shape) * data.sum() ** 2 / data.nnz), 'hs')
# Uinit = kr_balance_factors(Unew1)

In [ ]:
%autoreload
Unew2_list = []
cost_arr_list = []
deltas = np.logspace(-6, -3, 4)
eps = 1e-4
print(deltas)
print(sigma)
# sigma = deltas[-1] * np.sqrt(data.nnz)
for delta in deltas:
    print("delta = ", delta)
    Unew2, cost_arr = tensor_completion_maxnorm(data, 4 * r**t, delta, epsilon=eps,
                                                #sgd=True, sgd_batch_size=2000,
                                                #U0 = kr_balance_factors(Unew1),
                                                init='svdrand',
                                                kappa=100, beta=1, 
                                                tol=1e-10, inner_tol=1e-12, max_iter=300, inner_max_iter=10, 
                                                verbosity=1, inner_line_iter=40,
                                                rebalance=True)
    Unew2_list.append(Unew2)
    cost_arr_list.append(cost_arr)


[1.e-06 1.e-05 1.e-04 1.e-03]
0.005
delta =  1e-06
[1.e-06 1.e-05 1.e-04 1.e-03]
0.005
delta =  1e-06
Initial cost: 2.040e+06
Initial qnorm_ub: 9.450e+03
|| r || = 1.022e+00, delta = 5.072e-09
Initial cost: 2.040e+06
Initial qnorm_ub: 9.450e+03
|| r || = 1.022e+00, delta = 5.072e-09

Iteration 0 complete


scaled || r || = 3.408e-01, delta = 5.072e-09
Max-qnorm upper bound: 3.134e+05
Cost function:         5.391e+05



Iteration 0 complete


scaled || r || = 3.408e-01, delta = 5.072e-09
Max-qnorm upper bound: 3.134e+05
Cost function:         5.391e+05



Iteration 1 complete


scaled || r || = 1.519e-01, delta = 5.072e-09
Max-qnorm upper bound: 1.915e+05
Cost function:         2.363e+05



Iteration 1 complete


scaled || r || = 1.519e-01, delta = 5.072e-09
Max-qnorm upper bound: 1.915e+05
Cost function:         2.363e+05



Iteration 2 complete


scaled || r || = 1.094e-01, delta = 5.072e-09
Max-qnorm upper bound: 1.510e+05
Cost function:         1.743e+05



Iteration 2 complete


sc


Iteration 20 complete


scaled || r || = 3.537e-02, delta = 5.072e-09
Max-qnorm upper bound: 3.893e+04
Cost function:         4.137e+04



Iteration 20 complete


scaled || r || = 3.537e-02, delta = 5.072e-09
Max-qnorm upper bound: 3.893e+04
Cost function:         4.137e+04



Iteration 21 complete


scaled || r || = 3.440e-02, delta = 5.072e-09
Max-qnorm upper bound: 3.766e+04
Cost function:         3.996e+04



Iteration 21 complete


scaled || r || = 3.440e-02, delta = 5.072e-09
Max-qnorm upper bound: 3.766e+04
Cost function:         3.996e+04



Iteration 22 complete


scaled || r || = 3.349e-02, delta = 5.072e-09
Max-qnorm upper bound: 3.646e+04
Cost function:         3.864e+04



Iteration 22 complete


scaled || r || = 3.349e-02, delta = 5.072e-09
Max-qnorm upper bound: 3.646e+04
Cost function:         3.864e+04



Iteration 23 complete


scaled || r || = 3.267e-02, delta = 5.072e-09
Max-qnorm upper bound: 3.534e+04
Cost function:         3.741e+04



Iteration 23 complete


sc


Iteration 41 complete


scaled || r || = 2.308e-02, delta = 5.072e-09
Max-qnorm upper bound: 2.289e+04
Cost function:         2.392e+04



Iteration 41 complete


scaled || r || = 2.308e-02, delta = 5.072e-09
Max-qnorm upper bound: 2.289e+04
Cost function:         2.392e+04



Iteration 42 complete


scaled || r || = 2.272e-02, delta = 5.072e-09
Max-qnorm upper bound: 2.248e+04
Cost function:         2.348e+04



Iteration 42 complete


scaled || r || = 2.272e-02, delta = 5.072e-09
Max-qnorm upper bound: 2.248e+04
Cost function:         2.348e+04



Iteration 43 complete


scaled || r || = 2.242e-02, delta = 5.072e-09
Max-qnorm upper bound: 2.209e+04
Cost function:         2.307e+04



Iteration 43 complete


scaled || r || = 2.242e-02, delta = 5.072e-09
Max-qnorm upper bound: 2.209e+04
Cost function:         2.307e+04



Iteration 44 complete


scaled || r || = 2.212e-02, delta = 5.072e-09
Max-qnorm upper bound: 2.172e+04
Cost function:         2.267e+04



Iteration 44 complete


sc


Iteration 62 complete


scaled || r || = 1.824e-02, delta = 5.072e-09
Max-qnorm upper bound: 1.669e+04
Cost function:         1.734e+04



Iteration 62 complete


scaled || r || = 1.824e-02, delta = 5.072e-09
Max-qnorm upper bound: 1.669e+04
Cost function:         1.734e+04



Iteration 63 complete


scaled || r || = 1.808e-02, delta = 5.072e-09
Max-qnorm upper bound: 1.648e+04
Cost function:         1.712e+04



Iteration 63 complete


scaled || r || = 1.808e-02, delta = 5.072e-09
Max-qnorm upper bound: 1.648e+04
Cost function:         1.712e+04



Iteration 64 complete


scaled || r || = 1.793e-02, delta = 5.072e-09
Max-qnorm upper bound: 1.628e+04
Cost function:         1.691e+04



Iteration 64 complete


scaled || r || = 1.793e-02, delta = 5.072e-09
Max-qnorm upper bound: 1.628e+04
Cost function:         1.691e+04



Iteration 65 complete


scaled || r || = 1.778e-02, delta = 5.072e-09
Max-qnorm upper bound: 1.608e+04
Cost function:         1.670e+04



Iteration 65 complete


sc


Iteration 83 complete


scaled || r || = 1.587e-02, delta = 5.072e-09
Max-qnorm upper bound: 1.325e+04
Cost function:         1.374e+04



Iteration 83 complete


scaled || r || = 1.587e-02, delta = 5.072e-09
Max-qnorm upper bound: 1.325e+04
Cost function:         1.374e+04



Iteration 84 complete


scaled || r || = 1.579e-02, delta = 5.072e-09
Max-qnorm upper bound: 1.313e+04
Cost function:         1.361e+04



Iteration 84 complete


scaled || r || = 1.579e-02, delta = 5.072e-09
Max-qnorm upper bound: 1.313e+04
Cost function:         1.361e+04



Iteration 85 complete


scaled || r || = 1.571e-02, delta = 5.072e-09
Max-qnorm upper bound: 1.300e+04
Cost function:         1.348e+04



Iteration 85 complete


scaled || r || = 1.571e-02, delta = 5.072e-09
Max-qnorm upper bound: 1.300e+04
Cost function:         1.348e+04



Iteration 86 complete


scaled || r || = 1.561e-02, delta = 5.072e-09
Max-qnorm upper bound: 1.289e+04
Cost function:         1.336e+04



Iteration 86 complete


sc

...


Iteration 17 complete


scaled || r || = 4.306e-02, delta = 5.072e-07
Max-qnorm upper bound: 4.234e+04
Cost function:         4.595e+04



Iteration 17 complete


scaled || r || = 4.306e-02, delta = 5.072e-07
Max-qnorm upper bound: 4.234e+04
Cost function:         4.595e+04



Iteration 18 complete


scaled || r || = 4.199e-02, delta = 5.072e-07
Max-qnorm upper bound: 4.075e+04
Cost function:         4.418e+04



Iteration 18 complete


scaled || r || = 4.199e-02, delta = 5.072e-07
Max-qnorm upper bound: 4.075e+04
Cost function:         4.418e+04



Iteration 19 complete


scaled || r || = 4.094e-02, delta = 5.072e-07
Max-qnorm upper bound: 3.920e+04
Cost function:         4.246e+04



Iteration 19 complete


scaled || r || = 4.094e-02, delta = 5.072e-07
Max-qnorm upper bound: 3.920e+04
Cost function:         4.246e+04



Iteration 20 complete


scaled || r || = 3.998e-02, delta = 5.072e-07
Max-qnorm upper bound: 3.773e+04
Cost function:         4.084e+04



Iteration 20 complete


sc


Iteration 38 complete


scaled || r || = 2.804e-02, delta = 5.072e-07
Max-qnorm upper bound: 2.216e+04
Cost function:         2.369e+04



Iteration 38 complete


scaled || r || = 2.804e-02, delta = 5.072e-07
Max-qnorm upper bound: 2.216e+04
Cost function:         2.369e+04



Iteration 39 complete


scaled || r || = 2.767e-02, delta = 5.072e-07
Max-qnorm upper bound: 2.166e+04
Cost function:         2.315e+04



Iteration 39 complete


scaled || r || = 2.767e-02, delta = 5.072e-07
Max-qnorm upper bound: 2.166e+04
Cost function:         2.315e+04



Iteration 40 complete


scaled || r || = 2.725e-02, delta = 5.072e-07
Max-qnorm upper bound: 2.122e+04
Cost function:         2.266e+04



Iteration 40 complete


scaled || r || = 2.725e-02, delta = 5.072e-07
Max-qnorm upper bound: 2.122e+04
Cost function:         2.266e+04



Iteration 41 complete


scaled || r || = 2.690e-02, delta = 5.072e-07
Max-qnorm upper bound: 2.075e+04
Cost function:         2.216e+04



Iteration 41 complete


sc


Iteration 59 complete


scaled || r || = 2.203e-02, delta = 5.072e-07
Max-qnorm upper bound: 1.550e+04
Cost function:         1.645e+04



Iteration 59 complete


scaled || r || = 2.203e-02, delta = 5.072e-07
Max-qnorm upper bound: 1.550e+04
Cost function:         1.645e+04



Iteration 60 complete


scaled || r || = 2.181e-02, delta = 5.072e-07
Max-qnorm upper bound: 1.529e+04
Cost function:         1.622e+04



Iteration 60 complete


scaled || r || = 2.181e-02, delta = 5.072e-07
Max-qnorm upper bound: 1.529e+04
Cost function:         1.622e+04



Iteration 61 complete


scaled || r || = 2.160e-02, delta = 5.072e-07
Max-qnorm upper bound: 1.509e+04
Cost function:         1.600e+04



Iteration 61 complete


scaled || r || = 2.160e-02, delta = 5.072e-07
Max-qnorm upper bound: 1.509e+04
Cost function:         1.600e+04



Iteration 62 complete


scaled || r || = 2.139e-02, delta = 5.072e-07
Max-qnorm upper bound: 1.489e+04
Cost function:         1.578e+04



Iteration 62 complete


sc


Iteration 80 complete


scaled || r || = 1.835e-02, delta = 5.072e-07
Max-qnorm upper bound: 1.199e+04
Cost function:         1.264e+04



Iteration 80 complete


scaled || r || = 1.835e-02, delta = 5.072e-07
Max-qnorm upper bound: 1.199e+04
Cost function:         1.264e+04



Iteration 81 complete


scaled || r || = 1.822e-02, delta = 5.072e-07
Max-qnorm upper bound: 1.186e+04
Cost function:         1.250e+04



Iteration 81 complete


scaled || r || = 1.822e-02, delta = 5.072e-07
Max-qnorm upper bound: 1.186e+04
Cost function:         1.250e+04



Iteration 82 complete


scaled || r || = 1.809e-02, delta = 5.072e-07
Max-qnorm upper bound: 1.173e+04
Cost function:         1.236e+04



Iteration 82 complete


scaled || r || = 1.809e-02, delta = 5.072e-07
Max-qnorm upper bound: 1.173e+04
Cost function:         1.236e+04



Iteration 83 complete


scaled || r || = 1.796e-02, delta = 5.072e-07
Max-qnorm upper bound: 1.160e+04
Cost function:         1.223e+04



Iteration 83 complete


sc


Iteration 101 complete


scaled || r || = 1.609e-02, delta = 5.072e-07
Max-qnorm upper bound: 9.743e+03
Cost function:         1.025e+04



Iteration 101 complete


scaled || r || = 1.609e-02, delta = 5.072e-07
Max-qnorm upper bound: 9.743e+03
Cost function:         1.025e+04



Iteration 102 complete


scaled || r || = 1.601e-02, delta = 5.072e-07
Max-qnorm upper bound: 9.658e+03
Cost function:         1.016e+04



Iteration 102 complete


scaled || r || = 1.601e-02, delta = 5.072e-07
Max-qnorm upper bound: 9.658e+03
Cost function:         1.016e+04



Iteration 103 complete


scaled || r || = 1.592e-02, delta = 5.072e-07
Max-qnorm upper bound: 9.574e+03
Cost function:         1.007e+04



Iteration 103 complete


scaled || r || = 1.592e-02, delta = 5.072e-07
Max-qnorm upper bound: 9.574e+03
Cost function:         1.007e+04



Iteration 104 complete


scaled || r || = 1.584e-02, delta = 5.072e-07
Max-qnorm upper bound: 9.494e+03
Cost function:         9.981e+03



Iteration 104 compl

In [ ]:
qnorm_max = max_qnorm_ub(Unew2)
print(qnorm_max)

In [ ]:
expander_eigs = np.abs(nx.linalg.adjacency_spectrum(expander))
expander_eigs.sort()
print(expander_eigs[-10:])

In [ ]:
bound = 2**(2*t-4) * 1.8**(t-1) * (qnorm_max + qnorm_true)**2 * ((1 + expander_eigs[-2]/expander_eigs[-1])**(t-1) - 1)
print(bound)
print(bound / n**t)

In [ ]:
for i in range(len(Unew2_list)):
    print("delta = {}".format(deltas[i]))
    print("loss of U true:     %.3e" % np.sqrt(loss(U, data) / data.nnz))
    print("loss of U max:      %.3e\n" % np.sqrt(loss(Unew2_list[i], data) / data.nnz))

In [ ]:
import matplotlib.pyplot as plt

plt.semilogy(cost_arr_list[0])
plt.semilogy(cost_arr_list[1])
plt.semilogy(cost_arr_list[2])
plt.semilogy(cost_arr_list[3])
# plt.semilogy(cost_arr_list[4])
# plt.semilogy(cost_arr_list[5])

plt.legend(["delta = 1e-6", "delta = 1e-5", "delta = 1e-4", "delta = 1e-3"])
plt.xlabel('iterate')
plt.ylabel('cost')
plt.title("Cost-Curve for Various Delta Values")
plt.savefig('Cost-Curve-Delta_300.png')

## Generalization error

In [ ]:
def gen_err(Upred, Utrue):
    norm_true = kr_dot(Utrue, Utrue)
    mse_gen = kr_dot(Upred, Upred) + norm_true - 2 * kr_dot(Upred, Utrue)
    return np.sqrt(mse_gen / norm_true)

def mse_gen_err(Upred, Utrue):
    norm_true = kr_dot(Utrue, Utrue)
    mse_gen = kr_dot(Upred, Upred) + norm_true - 2 * kr_dot(Upred, Utrue)
    return np.sqrt(mse_gen / Upred[0].shape[0] ** len(Upred))


for i in range(len(Unew2_list)):
    print("delta = {}".format(deltas[i]))
    print("relative RMSE max:      %1.4e" % gen_err(Unew2_list[i], U))
    print("MSE max:                %1.4e\n" % mse_gen_err(Unew2_list[i], U))

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, axs = plt.subplots(1,t, figsize=(6, 20))
for i in range(t):
    im = axs[i].imshow(Unew2[i])
    divider = make_axes_locatable(axs[i])
    cax = divider.append_axes('right', size='5%', pad=0.05)
    fig.colorbar(im, cax=cax, orientation='vertical')
    if i > 0:
        axs[i].set_yticks([])

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, axs = plt.subplots(1,t, figsize=(6, 20))
for i in range(t):
    im = axs[i].imshow(U[i])
    divider = make_axes_locatable(axs[i])
    cax = divider.append_axes('right', size='5%', pad=0.05)
    fig.colorbar(im, cax=cax, orientation='vertical')
    if i > 0:
        axs[i].set_yticks([])